## Importing Libraries

In [8]:
# Matrix and Dataframe Operations
import pandas as pd
import numpy as np
# Tensorflow Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
import plotly
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from matplotlib.ticker import FormatStrFormatter
import plotly.graph_objects as go

### Required Function (Pickle Save and Load)

In [9]:
import pickle
def save_pickle(data,path):
    with open(path, 'wb') as file:
        pickle.dump(data, file, pickle.HIGHEST_PROTOCOL)

def load_pickle(path):
    with open(path, 'rb') as file:
        result = pickle.load(file)
    return result

## Reading TFRecords (Training and Validation datasets)

In [ ]:
# Iterate over the whole dataset to count records/samples (https://www.rustyrobotics.com/posts/tensorflow/tfdataset-record-count/)
def countRecords(ds:tf.data.Dataset):
    count = 0
    if tf.executing_eagerly():
        # TF v2 or v1 in eager mode
        for r in ds:
            count = count+1
    else:
        # TF v1 in non-eager mode
        iterator = tf.compat.v1.data.make_one_shot_iterator(ds)
        next_batch = iterator.get_next()
        with tf.compat.v1.Session() as sess:
            try:
                while True:
                    sess.run(next_batch)
                    count = count+1
            except tf.errors.OutOfRangeError:
                pass
    return count

In [10]:
def read_tfrecord(serialized_example, export_subject=False):
    tfrecord_format = (
        {
            'label': tf.io.FixedLenFeature([], tf.int64),
            'feature': tf.io.FixedLenFeature([300], tf.float32),
            'subject': tf.io.FixedLenFeature([], tf.int64),
            'burst':  tf.io.FixedLenFeature([], tf.int64)
        }
    )
    example = tf.io.parse_single_example(serialized_example, tfrecord_format)
    f = tf.reshape(example['feature'], [window,1])
    f.set_shape([window, 1])
    # One-hot encode the label to match the expected shape for categorical_crossentropy
    label = tf.one_hot(example['label'], depth=14)
    if export_subject:
        return f, label, example['subject']
    return f, label

def get_dataset(tf_record_name):
    dataset = tf.data.TFRecordDataset(tf_record_name)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    dataset_samples = countRecords(dataset)
#     print("Samples: ", dataset_samples)
    dataset = dataset.shuffle(dataset_samples)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
window = 300
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 1024
train_dataset = get_dataset('all_mixed_train.tfrecord')
valid_dataset = get_dataset('all_mixed_val.tfrecord')

In [ ]:
for feature, label in train_dataset.take(10):
    print('label={}, feature={}'.format(label.shape, feature.shape))

# CNN Layer 1: Layer-Wise-Unsupervised Pretraining

In [13]:
# Architecture of CNN1:

# Parameters
conv1D_1 = [32,5,1]  # number_filters,kernel_size and strides 
pool1D_1 = [2,2]     # pool_size and strides
activation = "leakyrelu"

# Definition
model1 = Sequential()
model1.add(InputLayer((window,1))) #InputLayer(BURST_WINDOW, N_CHANNELS)

# CNN LAYER 1: DOWN BRANCH
model1.add(Conv1D(filters=conv1D_1[0], kernel_size=conv1D_1[1], # TODO TRY WITH HIGHER KERNEL SIZE (ODD NUMBER!)
                  strides=conv1D_1[2], padding='same', name='cnn_layer_1')) # activation='relu'))

model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling1D(pool_size=pool1D_1[0], strides=pool1D_1[1], padding='same'))

# CNN LAYER 1: UP BRANCH
model1.add(Conv1DTranspose(filters=1, kernel_size=conv1D_1[1], # TODO TRY WITH HIGHER KERNEL SIZE (ODD NUMBER!)
                           strides=conv1D_1[2]+1, #2
                           padding='same')) #activation='relu'))
model1.add(LeakyReLU(alpha=0.1))

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_layer_1 (Conv1D)        (None, 500, 32)           192       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 500, 32)           0         
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 250, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_transpose_1 (Conv1D  (None, 500, 1)            161       
 Transpose)                                                      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 500, 1)            0         
                                                                 
Total params: 353 (1.38 KB)
Trainable params: 353 (1.3